In [1]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

Initial Exploration of June 24th, 2017 data

In [2]:
## Setup

In [3]:
# imports a library 'pandas', names it as 'pd'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [4]:
print("Pandas version:",pd.__version__)
print("Numpy version:",np.__version__)

Pandas version: 0.20.2
Numpy version: 1.12.1


In [5]:
# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.options.display.float_format = '{:,.2f}'.format
#pd.set_option('display.precision',3)

In [6]:
# read data
cols = ['control_area', 'unit', 'scp', 'station', 'line_name',
        'division', 'date', 'time', 'desc','cum_entries', 'cum_exits']
df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt')
df.columns = cols
df.head()

,control_area,unit,scp,station,line_name,division,date,time,desc,cum_entries,cum_exits
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,00:00:00,REGULAR,6224816,2107317
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,04:00:00,REGULAR,6224850,2107322
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,08:00:00,REGULAR,6224885,2107352
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,12:00:00,REGULAR,6225005,2107452
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,16:00:00,REGULAR,6225248,2107513


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197416 entries, 0 to 197415
Data columns (total 11 columns):
control_area    197416 non-null object
unit            197416 non-null object
scp             197416 non-null object
station         197416 non-null object
line_name       197416 non-null object
division        197416 non-null object
date            197416 non-null object
time            197416 non-null object
desc            197416 non-null object
cum_entries     197416 non-null int64
cum_exits       197416 non-null int64
dtypes: int64(2), object(9)
memory usage: 16.6+ MB


In [8]:
# check contents of variables
print(df.shape)
print(df.desc.value_counts())
print(df.division.value_counts())

(197416, 11)
REGULAR       196958
RECOVR AUD       458
Name: desc, dtype: int64
IRT    72185
IND    70670
BMT    40102
PTH    12636
SRT     1399
RIT      424
Name: division, dtype: int64


In [9]:
# check turnstile's smallest set of uniquely identifiable variables
# result: a tunstile is unique by control area, scp and station

print(df[['control_area','scp','station']].drop_duplicates().shape)
print(df[['scp','station']].drop_duplicates().shape)
print(df[['control_area','scp','station','unit']].drop_duplicates().shape)


(4700, 3)
(4154, 2)
(4700, 4)


In [10]:
# add datetime variable
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format = '%m/%d/%Y %H:%M:%S')
df.head()

,control_area,unit,scp,station,line_name,division,date,time,desc,cum_entries,cum_exits,datetime
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,00:00:00,REGULAR,6224816,2107317,2017-06-17 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,04:00:00,REGULAR,6224850,2107322,2017-06-17 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,08:00:00,REGULAR,6224885,2107352,2017-06-17 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,12:00:00,REGULAR,6225005,2107452,2017-06-17 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,16:00:00,REGULAR,6225248,2107513,2017-06-17 16:00:00


In [11]:
# add shifted data
df[['shift_cum_entries','shift_cum_exits', 'shift_datetime']] = \
                         (df
                         .sort_values(['control_area','scp','station','datetime'])
                         .groupby(['control_area','scp', 'station'])[['cum_entries','cum_exits','datetime']]
                         .shift(1))
    
df['entries'] = df['cum_entries'] - df['shift_cum_entries']
df['exits'] = df['cum_exits'] - df['shift_cum_exits']

#df.iloc[-50:]
df[['entries','exits']].to_csv('example.csv')

In [12]:
df[['entries','exits']].describe()


,entries,exits
count,"192,716.00","192,716.00"
mean,"17,411.97","5,104.51"
std,"5,145,724.07","1,830,591.39"
min,"-5,449,929.00","-2,335,317.00"
25%,11.00,8.00
50%,80.00,54.00
75%,252.00,170.00
max,"1,621,023,313.00","785,938,643.00"


In [13]:
# describe negative/zero values
print('percentage of negative entries', "{0:.3f}%".format(df[df.entries < 0].shape[0]/df.shape[0]))
print('percentage of zero entries', "{0:.3f}%".format(df[df.entries == 0].shape[0]/df.shape[0]))
print('percentage of negative exits', "{0:.3f}%".format(df[df.exits < 0].shape[0]/df.shape[0]))
print('percentage of zero entries', "{0:.3f}%".format(df[df.exits == 0].shape[0]/df.shape[0]))

# - since negative percentage is small, we can take a look at the negative numbers to tell how the numbers are wrong
# - the zero recordings are likely to be from the station closing
# - we can manually get a sense the outlieres of positive numbers by eyeballing the numbers


percentage of negative entries 0.008%
percentage of zero entries 0.142%
percentage of negative exits 0.006%
percentage of zero entries 0.140%


In [14]:
# examples of data investigation - draft

df[df.entries == -5449929.00]
#ranges = [-6000000,0,11,80,252,50,60,70,80,90,100]
#df.groupby(pd.cut(df.a, ranges)).count()
#np.log(df['entries'])
df.iloc[131914-10:131914+10]
df[df.entries == 1621023313.00]


,control_area,unit,scp,station,line_name,division,date,time,desc,cum_entries,cum_exits,datetime,shift_cum_entries,shift_cum_exits,shift_datetime,entries,exits
29158,H008,R248,01-00-04,1 AV,L,BMT,06/18/2017,16:00:00,REGULAR,1627413160,167780724,2017-06-18 16:00:00,"6,389,847.00","674,838.00",2017-06-18 12:00:00,"1,621,023,313.00","167,105,886.00"


In [15]:
# examples of data investigation - draft

df.iloc[29158-10:29158+10]
df[(df.time == '16:00:00') & (df.scp == '01-00-04') & (df.control_area == 'H008') & (df.station == '1 AV')]

,control_area,unit,scp,station,line_name,division,date,time,desc,cum_entries,cum_exits,datetime,shift_cum_entries,shift_cum_exits,shift_datetime,entries,exits
29152,H008,R248,01-00-04,1 AV,L,BMT,06/17/2017,16:00:00,REGULAR,6388172,674664,2017-06-17 16:00:00,"6,387,674.00","674,582.00",2017-06-17 12:00:00,498.00,82.00
29158,H008,R248,01-00-04,1 AV,L,BMT,06/18/2017,16:00:00,REGULAR,1627413160,167780724,2017-06-18 16:00:00,"6,389,847.00","674,838.00",2017-06-18 12:00:00,"1,621,023,313.00","167,105,886.00"
29164,H008,R248,01-00-04,1 AV,L,BMT,06/19/2017,16:00:00,REGULAR,1627416511,167781177,2017-06-19 16:00:00,"1,627,415,718.00","167,781,103.00",2017-06-19 12:00:00,793.00,74.00
29170,H008,R248,01-00-04,1 AV,L,BMT,06/20/2017,16:00:00,REGULAR,1627420314,167781671,2017-06-20 16:00:00,"1,627,419,495.00","167,781,602.00",2017-06-20 12:00:00,819.00,69.00
29176,H008,R248,01-00-04,1 AV,L,BMT,06/21/2017,16:00:00,REGULAR,1627424460,167782077,2017-06-21 16:00:00,"1,627,423,678.00","167,782,007.00",2017-06-21 12:00:00,782.00,70.00
29182,H008,R248,01-00-04,1 AV,L,BMT,06/22/2017,16:00:00,REGULAR,1627428908,167782483,2017-06-22 16:00:00,"1,627,428,091.00","167,782,420.00",2017-06-22 12:00:00,817.00,63.00
29188,H008,R248,01-00-04,1 AV,L,BMT,06/23/2017,16:00:00,REGULAR,1627433325,167782906,2017-06-23 16:00:00,"1,627,432,427.00","167,782,828.00",2017-06-23 12:00:00,898.00,78.00


In [ ]:
## next steps are 1. removing the outliers
## find a way to aggregate a station's
## then plot some basic facts
# - the most popular station by time-slot, by exit or entry
# - the most unpopular stations by time-slot, by exit or entry
# - total traffic (exit + entry)
